NDVI 仓库：输入是 时空栅格序列（H×W×channels），ConvLSTM 非常自然。

现在：是 单站点 8 个变量的时间序列，没有空间维度。为了仍然用 ConvLSTM，我采用了“把 8 个变量铺成 1×8 的网格”，让 ConvLSTM2D 可以吃进去，这属于把 NDVI 思路迁移到你的数据形态上。

如果想更“原汁原味”的 ConvLSTM（像 NDVI 那样有空间维），需要有空间网格数据；现在的数据更像“多变量时间序列”，LSTM/GRU 其实更合适，但要求 ConvLSTM，我们就用这个迁移做法。

## ConvLSTM
用 8 个变量铺成 1×8×1 的“伪空间网格”

输入 shape：(batch, time, 1, 8, 1)

输出：(batch, 1) 的 sm_30cm(t+1)


按时间顺序切分 train/val/test = 70%/15%/15%

用 train 训练，用 val 早停/调学习率

用 test 做指标评估（MAE / RMSE）

同时输出：每个区域最后日期的下一天（t+1）sm_30cm 预测值

必须保存：
1️⃣ 模型（ConvLSTM 权重 + 结构）
2️⃣ X 的 scaler
3️⃣ y 的 scaler

In [10]:
import os
import re
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Dict, Tuple

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks



In [11]:

# ============================================================
# Configuration
# ============================================================
# Input features (8 variables) and target (one-step ahead)
FEATURE_COLS = ["sm_30cm", "irrig_mm", "IRRAD", "TMIN", "TMAX", "VAP", "WIND", "RAIN"]
TARGET_COL = "sm_30cm"

# Column names in your CSV
DATE_COL = "date"
PROBE_COL = "probe_name"   # <-- confirmed by you

# Time-series supervised learning settings
LOOKBACK = 14              # use past 14 days to predict next day
TRAIN_RATIO = 0.70
VAL_RATIO = 0.15           # remaining 0.15 is test

# Training settings
EPOCHS = 150
BATCH_SIZE = 32
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)



In [12]:

# ============================================================
# Helpers
# ============================================================
def safe_name(s: str) -> str:
    """Make a string safe for filesystem paths."""
    s = str(s).strip()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^A-Za-z0-9_\-]+", "_", s)
    return s



In [13]:

def load_csv(csv_path: str) -> pd.DataFrame:
    """
    Load CSV and keep only required columns.
    This matches the 'data preparation' step used in the reference idea:
    clean -> sort by time -> build sequences.
    """
    df = pd.read_csv(csv_path)

    # Basic schema checks
    for col in [DATE_COL, PROBE_COL]:
        if col not in df.columns:
            raise ValueError(f"[{csv_path}] Missing required column: {col}")

    missing = [c for c in FEATURE_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"[{csv_path}] Missing feature columns: {missing}")

    # Parse dates and numeric columns
    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    df = df.dropna(subset=[DATE_COL]).copy()

    for c in FEATURE_COLS:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Keep only relevant columns
    df = df[[DATE_COL, PROBE_COL] + FEATURE_COLS].copy()

    # Sort to preserve time order within each probe (critical for time-series split)
    df = df.sort_values([PROBE_COL, DATE_COL]).reset_index(drop=True)
    return df



In [14]:
def clean_one_probe(df_probe: pd.DataFrame) -> pd.DataFrame:
    """
    Clean a single probe time series:
    - sort by date
    - time interpolation for missing values
    - forward/backward fill as fallback
    """
    df_probe = df_probe.sort_values(DATE_COL).copy()
    df_probe = df_probe.set_index(DATE_COL)

    df_probe[FEATURE_COLS] = df_probe[FEATURE_COLS].interpolate(
        method="time", limit_direction="both"
    )
    df_probe[FEATURE_COLS] = df_probe[FEATURE_COLS].ffill().bfill()

    df_probe = df_probe.reset_index()
    df_probe = df_probe.dropna(subset=FEATURE_COLS).reset_index(drop=True)
    return df_probe




In [15]:
def split_by_time(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Time-ordered split (no shuffling) into train/val/test = 70/15/15.
    This avoids leakage and matches standard forecasting practice.
    """
    n = len(df)
    n_train = int(n * TRAIN_RATIO)
    n_val = int(n * VAL_RATIO)

    train = df.iloc[:n_train].copy()
    val = df.iloc[n_train:n_train + n_val].copy()
    test = df.iloc[n_train + n_val:].copy()

    # Each split must be long enough to build sequences
    for name, part in [("train", train), ("val", val), ("test", test)]:
        if len(part) < LOOKBACK + 2:
            raise ValueError(
                f"Split '{name}' too short: {len(part)} rows. "
                f"Need at least LOOKBACK+2={LOOKBACK+2}. "
                f"Reduce LOOKBACK or use longer series."
            )

    return train, val, test




In [16]:
def make_supervised(
    df: pd.DataFrame,
    scaler_x: StandardScaler,
    scaler_y: StandardScaler,
    fit: bool
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Build supervised samples for one-step-ahead forecasting:
    X(t-LOOKBACK ... t-1) -> y(t)

    IMPORTANT: To reuse ConvLSTM from the reference repo's idea,
    we reshape the 8 tabular variables into a pseudo 2D grid (rows=1, cols=8),
    so ConvLSTM2D can process sequences of "frames" with shape (1, 8, 1).

    Output X shape:
      (samples, time=LOOKBACK, rows=1, cols=8, channels=1)
    """
    X_raw = df[FEATURE_COLS].values.astype(np.float32)
    y_raw = df[[TARGET_COL]].values.astype(np.float32)
    dates = df[DATE_COL].values  # aligned with y

    if fit:
        scaler_x.fit(X_raw)
        scaler_y.fit(y_raw)

    Xs = scaler_x.transform(X_raw)
    ys = scaler_y.transform(y_raw)

    X_list, y_list, d_list = [], [], []
    for i in range(LOOKBACK, len(df)):
        X_list.append(Xs[i - LOOKBACK:i, :])  # (LOOKBACK, 8)
        y_list.append(ys[i, 0])               # y at time i (scaled)
        d_list.append(dates[i])               # date of y

    X = np.array(X_list, dtype=np.float32)              # (N, LOOKBACK, 8)
    y = np.array(y_list, dtype=np.float32).reshape(-1, 1)
    d = np.array(d_list)

    # Reshape tabular features into pseudo image for ConvLSTM2D
    X = X.reshape(X.shape[0], LOOKBACK, 1, len(FEATURE_COLS), 1)
    return X, y, d




In [17]:
def build_convlstm(lookback: int, width: int) -> tf.keras.Model:
    """
    ConvLSTM backbone (inspired by the reference repo's ConvLSTM forecasting idea).
    """
    inp = layers.Input(shape=(lookback, 1, width, 1))

    x = layers.ConvLSTM2D(
        filters=16,
        kernel_size=(1, 3),
        padding="same",
        return_sequences=True,
        activation="tanh",
    )(inp)
    x = layers.BatchNormalization()(x)

    x = layers.ConvLSTM2D(
        filters=16,
        kernel_size=(1, 3),
        padding="same",
        return_sequences=False,
        activation="tanh",
    )(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    out = layers.Dense(1, activation="linear")(x)

    model = models.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
    return model


def inverse_y(scaler_y: StandardScaler, y_scaled_2d: np.ndarray) -> np.ndarray:
    """Inverse transform scaled y back to original units."""
    return scaler_y.inverse_transform(y_scaled_2d).reshape(-1)


def plot_loss(history: tf.keras.callbacks.History, out_png: str, title: str) -> None:
    """Plot train/validation loss curves (single figure)."""
    plt.figure()
    plt.plot(history.history["loss"], label="train_loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.xlabel("epoch")
    plt.ylabel("loss (MSE on scaled y)")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()


def plot_test_true_vs_pred(dates, y_true, y_pred, out_png: str, title: str) -> None:
    """Plot test series: actual vs prediction (like the example screenshot)."""
    plt.figure()
    plt.plot(dates, y_true, label="actual")
    plt.plot(dates, y_pred, label="prediction")
    plt.xlabel("date")
    plt.ylabel("sm_30cm (m3/m3)")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()




In [18]:
# ============================================================
# Training / Evaluation per probe
# ============================================================
def train_eval_predict_one_probe(region: str, probe: str, df_probe: pd.DataFrame, out_root: str = "outputs") -> None:
    """
    For each probe (sub-region):
    - Clean data
    - Time split 70/15/15
    - Train ConvLSTM
    - Save model + scalers + history CSV
    - Plot loss curve and test true-vs-pred curve
    - Report test metrics and next-day prediction
    """
    df_probe = clean_one_probe(df_probe)

    # minimal sanity check
    if len(df_probe) < LOOKBACK + 10:
        print(f"[SKIP] {region}/{probe} too short: {len(df_probe)} rows")
        return

    train_df, val_df, test_df = split_by_time(df_probe)

    scaler_x = StandardScaler()
    scaler_y = StandardScaler()

    X_train, y_train, _ = make_supervised(train_df, scaler_x, scaler_y, fit=True)
    X_val, y_val, _ = make_supervised(val_df, scaler_x, scaler_y, fit=False)
    X_test, y_test, d_test = make_supervised(test_df, scaler_x, scaler_y, fit=False)

    model = build_convlstm(LOOKBACK, len(FEATURE_COLS))

    # Output directory per (region, probe)
    probe_dir = os.path.join(out_root, safe_name(region), safe_name(probe))
    os.makedirs(probe_dir, exist_ok=True)

    model_path = os.path.join(probe_dir, "model.h5")
    scaler_x_path = os.path.join(probe_dir, "scaler_x.pkl")
    scaler_y_path = os.path.join(probe_dir, "scaler_y.pkl")
    history_csv = os.path.join(probe_dir, "history.csv")
    loss_png = os.path.join(probe_dir, "loss_train_val.png")
    test_png = os.path.join(probe_dir, "test_true_vs_pred.png")
    test_csv = os.path.join(probe_dir, "test_compare.csv")

    # Callbacks:
    # - ModelCheckpoint: keep best weights based on val_loss
    # - EarlyStopping: stop when no improvement
    # - ReduceLROnPlateau: stabilize optimization
    # - CSVLogger: store per-epoch loss/val_loss
    cbs = [
        callbacks.ModelCheckpoint(model_path, monitor="val_loss", save_best_only=True),
        callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=1e-5),
        callbacks.CSVLogger(history_csv, append=False),
    ]

    print(f"\n=== TRAINING: {region} / {probe} ===")
    print(f"Samples: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}  (lookback={LOOKBACK})")

    # verbose=1 prints train/val loss each epoch, as you requested
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=cbs,
        verbose=1
    )

    # Save scalers to guarantee consistent normalization at inference time
    joblib.dump(scaler_x, scaler_x_path)
    joblib.dump(scaler_y, scaler_y_path)

    # Plot train/val loss (single figure)
    plot_loss(history, loss_png, title=f"Loss: {region}/{probe}")

    # ------------------------------------------------------------
    # Test evaluation (inverse transform to original units)
    # ------------------------------------------------------------
    y_pred_test_scaled = model.predict(X_test, verbose=0)     # scaled predictions
    y_true = inverse_y(scaler_y, y_test)                      # original units
    y_pred = inverse_y(scaler_y, y_pred_test_scaled)          # original units

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Percentage error (MAPE) to quantify "prediction vs true in percent"
    denom = np.maximum(np.abs(y_true), 1e-8)
    mape = float(np.mean(np.abs((y_pred - y_true) / denom)) * 100.0)

    print(f"[TEST] MAE={mae:.6f} RMSE={rmse:.6f} MAPE={mape:.2f}%  (unit: m3/m3)")

    # Save test comparison table (date, true, pred, percent error)
    pct_err = (y_pred - y_true) / denom * 100.0
    cmp_df = pd.DataFrame({
        "date": pd.to_datetime(d_test),
        "y_true_sm_30cm": y_true,
        "y_pred_sm_30cm": y_pred,
        "pct_error_%": pct_err
    })
    cmp_df.to_csv(test_csv, index=False)

    # Plot test curve (actual vs prediction)
    plot_test_true_vs_pred(pd.to_datetime(d_test), y_true, y_pred, test_png,
                           title=f"Test True vs Pred: {region}/{probe}")

    # ------------------------------------------------------------
    # Next-day prediction: last available date + 1 day
    # ------------------------------------------------------------
    last_dt = pd.to_datetime(df_probe[DATE_COL].iloc[-1])
    next_dt = last_dt + pd.Timedelta(days=1)

    last_window = df_probe[FEATURE_COLS].iloc[-LOOKBACK:].values.astype(np.float32)
    X_last = scaler_x.transform(last_window).reshape(1, LOOKBACK, 1, len(FEATURE_COLS), 1)

    y_next_scaled = model.predict(X_last, verbose=0)
    y_next = scaler_y.inverse_transform(y_next_scaled)[0, 0]

    print(f"[NEXT DAY] last_date={last_dt.date()} -> predict_date={next_dt.date()}  "
          f"pred_sm_30cm={y_next:.6f} (m3/m3)")
    print(f"Saved outputs to: {probe_dir}")




In [19]:
# ============================================================
# Main: 4 CSV files -> split by probe_name -> train/eval each
# ============================================================
if __name__ == "__main__":
    REGION_CSVS: Dict[str, str] = {
        "Grandvillers_Sec": r"D:\UV Projet\Soil Moisture\Grandvillers_Sec.csv",
        "Grandvillers_Canon": r"D:\UV Projet\Soil Moisture\Grandvillers-Canon.csv",
        "Grandvillers_Robot_20": r"D:\UV Projet\Soil Moisture\Grandvillers-Robot-20.csv",
        "Grandvillers_Robot": r"D:\UV Projet\Soil Moisture\Grandvillers-Robot.csv",
    }

    all_probes = set()

    for region, path in REGION_CSVS.items():
        if not os.path.exists(path):
            print(f"[SKIP] File not found: {path}")
            continue

        df = load_csv(path)

        probes = sorted(df[PROBE_COL].dropna().unique().tolist())
        all_probes.update(probes)

        print(f"\nRegion={region}  probes={probes}  (count={len(probes)})")

        for probe in probes:
            df_probe = df[df[PROBE_COL] == probe].copy()
            train_eval_predict_one_probe(region, probe, df_probe, out_root="outputs")

    print("\nAll unique probe_name values across 4 CSV:")
    print(sorted(list(all_probes)))
    print(f"Total unique probe_name: {len(all_probes)}")



Region=Grandvillers_Sec  probes=['Sec']  (count=1)

=== TRAINING: Grandvillers_Sec / Sec ===
Samples: train=66, val=3, test=4  (lookback=14)
Epoch 1/150
3/3 [==============================] - 0s 77ms/step - loss: 2.0923 - val_loss: 0.3485 - lr: 0.0010
Epoch 2/150
3/3 [==============================] - 0s 19ms/step - loss: 0.8616 - val_loss: 0.3540 - lr: 0.0010
Epoch 3/150
3/3 [==============================] - 0s 16ms/step - loss: 0.6797 - val_loss: 0.3602 - lr: 0.0010
Epoch 4/150
3/3 [==============================] - 0s 15ms/step - loss: 0.4922 - val_loss: 0.3627 - lr: 0.0010
Epoch 5/150
3/3 [==============================] - 0s 14ms/step - loss: 0.3383 - val_loss: 0.3696 - lr: 0.0010
Epoch 6/150
3/3 [==============================] - 0s 14ms/step - loss: 0.4168 - val_loss: 0.3745 - lr: 0.0010
Epoch 7/150
3/3 [==============================] - 0s 14ms/step - loss: 0.2963 - val_loss: 0.3744 - lr: 0.0010
Epoch 8/150
3/3 [==============================] - 0s 15ms/step - loss: 0.2596 - 

##  在短期、深层土壤水分预测中，ConvLSTM 很容易退化为均值预测器 
输入是 (time, features)，没有空间维度

ConvLSTM 在这里 理论上不合适

原仓库用 ConvLSTM 是因为 NDVI 是栅格（H×W）

## 改进： 
1. 增大时间窗口：让模型看到“慢变量的累积效应”，而不是只看到短期噪声。

2. 预测多步（t+H）不是为了“看得更远”，
而是为了“逼模型必须学动态”，而不能靠惯性（persistence）混过去。

rows ≥ lookback + horizon + 1
要生成 1 条训练样本，你至少需要：

lookback 天作为输入

horizon 天作为预测目标

还要多 1 天，才能让时间索引对齐

那要怎么才能既保持 70/15/15，又跑 window=14/30/60 + multi-step？


✅ val/test “借用”前一段的 lookback 历史（不泄漏）

意思是：

val 的输入序列可以包含 train 最后 lookback 天（只当历史上下文）

但 val 的预测目标 y 仍然只在 val 区间

test 同理借 val（或 train+val）末尾 lookback 天

这样 val 不需要 ≥ lookback+horizon+1，只要 val ≥ horizon 就能跑。

这就是context stitching 修改方案。

这在时间序列里非常常见，因为预测时你本来也会用“过去的历史”作为上下文。

In [29]:
import os
import re
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Dict, Tuple, List

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks


# ============================================================
# Configuration
# ============================================================
FEATURE_COLS = ["sm_30cm", "irrig_mm", "IRRAD", "TMIN", "TMAX", "VAP", "WIND", "RAIN"]
TARGET_COL = "sm_30cm"

DATE_COL = "date"
PROBE_COL = "probe_name"   # confirmed

# Forecast horizon (multi-step): predict next H days
HORIZON = 7

# Compare different lookback windows
WINDOWS = [14, 30, 60]

# Time split ratios (must remain unchanged as requested)
TRAIN_RATIO = 0.70
VAL_RATIO = 0.15  # remaining 0.15 is test

EPOCHS = 150
BATCH_SIZE = 32
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)


# ============================================================
# Utility functions
# ============================================================
def safe_name(s: str) -> str:
    """Make a string safe for filesystem paths."""
    s = str(s).strip()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^A-Za-z0-9_\-]+", "_", s)
    return s


def load_csv(csv_path: str) -> pd.DataFrame:
    """Load CSV and keep required columns with basic type cleaning."""
    df = pd.read_csv(csv_path)

    for col in [DATE_COL, PROBE_COL]:
        if col not in df.columns:
            raise ValueError(f"[{csv_path}] Missing required column: {col}")

    missing = [c for c in FEATURE_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"[{csv_path}] Missing feature columns: {missing}")

    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    df = df.dropna(subset=[DATE_COL]).copy()

    for c in FEATURE_COLS:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df[[DATE_COL, PROBE_COL] + FEATURE_COLS].copy()
    df = df.sort_values([PROBE_COL, DATE_COL]).reset_index(drop=True)
    return df


def clean_one_probe(df_probe: pd.DataFrame) -> pd.DataFrame:
    """
    Clean a single probe series:
    - sort by date
    - interpolate by time
    - forward/backward fill
    """
    df_probe = df_probe.sort_values(DATE_COL).copy()
    df_probe = df_probe.set_index(DATE_COL)

    df_probe[FEATURE_COLS] = df_probe[FEATURE_COLS].interpolate(method="time", limit_direction="both")
    df_probe[FEATURE_COLS] = df_probe[FEATURE_COLS].ffill().bfill()

    df_probe = df_probe.reset_index()
    df_probe = df_probe.dropna(subset=FEATURE_COLS).reset_index(drop=True)
    return df_probe


def split_by_time(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Time-ordered split into train/val/test = 70/15/15 (no shuffling).
    """
    n = len(df)
    n_train = int(n * TRAIN_RATIO)
    n_val = int(n * VAL_RATIO)

    train = df.iloc[:n_train].copy()
    val = df.iloc[n_train:n_train + n_val].copy()
    test = df.iloc[n_train + n_val:].copy()
    return train, val, test


def require_train_min_length(train_df: pd.DataFrame, lookback: int, horizon: int):
    """
    For training, we cannot borrow context from the past outside train split.
    Therefore train must be long enough to build multi-step samples internally.
    """
    min_len = lookback + horizon + 1
    if len(train_df) < min_len:
        raise ValueError(
            f"Train split too short: {len(train_df)} rows. Need >= {min_len} "
            f"(lookback={lookback}, horizon={horizon})."
        )


def require_val_test_min_length(split_df: pd.DataFrame, horizon: int, split_name: str):
    """
    With context stitching, val/test only need to be long enough to provide targets.
    Minimum requirement: at least 'horizon' rows to form one target vector.
    """
    if len(split_df) < horizon:
        raise ValueError(
            f"{split_name} split too short: {len(split_df)} rows. Need >= {horizon} (horizon={horizon})."
        )


def make_supervised_multistep(
    df_all: pd.DataFrame,
    lookback: int,
    horizon: int,
    scaler_x: StandardScaler,
    scaler_y: StandardScaler,
    fit: bool
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Build multi-step supervised samples from a single DataFrame (used for train).
      X: past lookback days of 8 features
      y: next horizon days of sm_30cm

    Output:
      X shape: (N, lookback, 1, 8, 1)
      y shape: (N, horizon)
      d shape: forecast start date for each sample
    """
    X_raw = df_all[FEATURE_COLS].values.astype(np.float32)
    y_raw = df_all[[TARGET_COL]].values.astype(np.float32)
    dates = df_all[DATE_COL].values

    if fit:
        scaler_x.fit(X_raw)
        scaler_y.fit(y_raw)

    Xs = scaler_x.transform(X_raw)
    ys = scaler_y.transform(y_raw).reshape(-1)

    X_list, y_list, d_list = [], [], []
    for i in range(lookback, len(df_all) - horizon + 1):
        X_list.append(Xs[i - lookback:i, :])
        y_list.append(ys[i:i + horizon])
        d_list.append(dates[i])

    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list, dtype=np.float32)
    d = np.array(d_list)

    X = X.reshape(X.shape[0], lookback, 1, len(FEATURE_COLS), 1)
    return X, y, d


def make_supervised_multistep_with_context(
    df_context: pd.DataFrame,
    df_target: pd.DataFrame,
    lookback: int,
    horizon: int,
    scaler_x: StandardScaler,
    scaler_y: StandardScaler
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Context stitching (Solution A):
    - We allow using the last 'lookback' days from the previous split as input context
    - Targets (y) are strictly within df_target (no future leakage)

    df_context: typically last lookback rows from the previous split
    df_target : the split we are evaluating (val or test)

    Requirement for df_target:
      len(df_target) >= horizon  (enough target days)
    """
    # Concatenate context + target, preserving time order
    df_all = pd.concat([df_context, df_target], axis=0).copy()
    df_all = df_all.sort_values(DATE_COL).reset_index(drop=True)

    X_raw = df_all[FEATURE_COLS].values.astype(np.float32)
    y_raw = df_all[[TARGET_COL]].values.astype(np.float32)
    dates = df_all[DATE_COL].values

    Xs = scaler_x.transform(X_raw)
    ys = scaler_y.transform(y_raw).reshape(-1)

    ctx_len = len(df_context)

    X_list, y_list, d_list = [], [], []
    # i must be within the target part and y(i:i+horizon) must be fully in target
    for i in range(max(lookback, ctx_len), len(df_all) - horizon + 1):
        if i < ctx_len:
            continue
        if i + horizon - 1 < ctx_len:
            continue

        X_list.append(Xs[i - lookback:i, :])
        y_list.append(ys[i:i + horizon])
        d_list.append(dates[i])

    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list, dtype=np.float32)
    d = np.array(d_list)

    X = X.reshape(X.shape[0], lookback, 1, len(FEATURE_COLS), 1)
    return X, y, d


def build_convlstm_multistep(lookback: int, width: int, horizon: int) -> tf.keras.Model:
    """ConvLSTM model producing multi-step outputs (Dense(horizon))."""
    inp = layers.Input(shape=(lookback, 1, width, 1))

    x = layers.ConvLSTM2D(filters=16, kernel_size=(1, 3),
                          padding="same", return_sequences=True, activation="tanh")(inp)
    x = layers.BatchNormalization()(x)

    x = layers.ConvLSTM2D(filters=16, kernel_size=(1, 3),
                          padding="same", return_sequences=False, activation="tanh")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    out = layers.Dense(horizon, activation="linear")(x)

    model = models.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
    return model


def inverse_y_vector(scaler_y: StandardScaler, y_scaled: np.ndarray) -> np.ndarray:
    """Inverse-transform a 2D array (N, horizon) back to original units."""
    flat = y_scaled.reshape(-1, 1)
    inv = scaler_y.inverse_transform(flat).reshape(y_scaled.shape)
    return inv


def plot_loss(history: tf.keras.callbacks.History, out_png: str, title: str):
    """Plot train/val loss curves."""
    plt.figure()
    plt.plot(history.history["loss"], label="train_loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.xlabel("epoch")
    plt.ylabel("loss (MSE on scaled y)")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()


def plot_series(dates, y_true, y_pred, out_png: str, title: str, ylabel: str):
    """Plot actual vs prediction curves."""
    plt.figure()
    plt.plot(dates, y_true, label="actual")
    plt.plot(dates, y_pred, label="prediction")
    plt.xlabel("date")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()


def metrics_1d(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """Compute MAE, RMSE, and MAPE (%) on 1D arrays."""
    mae = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    denom = np.maximum(np.abs(y_true), 1e-8)
    mape = float(np.mean(np.abs((y_pred - y_true) / denom)) * 100.0)
    return {"MAE": mae, "RMSE": rmse, "MAPE_%": mape}


# ============================================================
# Experiment runner for one (region, probe, window)
# ============================================================
def run_one_setting(region: str, probe: str, df_probe: pd.DataFrame, lookback: int, horizon: int, out_root: str):
    """
    Train/evaluate one model for a specific lookback window:
    - Train samples are built strictly from train split
    - Val/test samples are built with context stitching (Solution A)
      * val uses last lookback rows from train as context
      * test uses last lookback rows from (train+val) as context
    """
    df_probe = clean_one_probe(df_probe)
    train_df, val_df, test_df = split_by_time(df_probe)

    # Minimal length checks
    require_train_min_length(train_df, lookback, horizon)
    require_val_test_min_length(val_df, horizon, "val")
    require_val_test_min_length(test_df, horizon, "test")

    # Fit scalers ONLY on train split (standard practice)
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()

    X_train, y_train, _ = make_supervised_multistep(
        df_all=train_df, lookback=lookback, horizon=horizon,
        scaler_x=scaler_x, scaler_y=scaler_y, fit=True
    )

    # Build val with context borrowed from end of train
    val_context = train_df.iloc[-lookback:].copy()
    X_val, y_val, _ = make_supervised_multistep_with_context(
        df_context=val_context, df_target=val_df,
        lookback=lookback, horizon=horizon,
        scaler_x=scaler_x, scaler_y=scaler_y
    )

    # Build test with context borrowed from end of (train+val)
    tv = pd.concat([train_df, val_df], axis=0).copy().sort_values(DATE_COL)
    test_context = tv.iloc[-lookback:].copy()
    X_test, y_test, d_test = make_supervised_multistep_with_context(
        df_context=test_context, df_target=test_df,
        lookback=lookback, horizon=horizon,
        scaler_x=scaler_x, scaler_y=scaler_y
    )

    if len(X_val) == 0 or len(X_test) == 0:
        raise ValueError(f"Not enough samples after context stitching: val={len(X_val)} test={len(X_test)}")

    model = build_convlstm_multistep(lookback, len(FEATURE_COLS), horizon)

    # Output folder
    out_dir = os.path.join(out_root, f"window_{lookback}", safe_name(region), safe_name(probe))
    os.makedirs(out_dir, exist_ok=True)

    model_path = os.path.join(out_dir, "model.h5")
    scaler_x_path = os.path.join(out_dir, "scaler_x.pkl")
    scaler_y_path = os.path.join(out_dir, "scaler_y.pkl")

    history_csv = os.path.join(out_dir, "history.csv")
    loss_png = os.path.join(out_dir, "loss_train_val.png")

    test_cmp_csv = os.path.join(out_dir, "test_compare_multistep.csv")
    test_step1_png = os.path.join(out_dir, "test_true_vs_pred_step1.png")
    test_stepH_png = os.path.join(out_dir, f"test_true_vs_pred_step{horizon}.png")

    last_forecast_csv = os.path.join(out_dir, f"last_date_forecast_next_{horizon}_days.csv")
    last_forecast_png = os.path.join(out_dir, f"last_date_forecast_next_{horizon}_days.png")

    summary_json = os.path.join(out_dir, "summary_metrics.json")

    cbs = [
        callbacks.ModelCheckpoint(model_path, monitor="val_loss", save_best_only=True),
        callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=1e-5),
        callbacks.CSVLogger(history_csv, append=False),
    ]

    print(f"\n=== TRAIN: window={lookback} horizon={horizon} | {region}/{probe} ===")
    print(f"Samples: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")
    # verbose=1 prints per-epoch loss/val_loss (your requirement)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=cbs,
        verbose=1
    )

    # Save scalers
    joblib.dump(scaler_x, scaler_x_path)
    joblib.dump(scaler_y, scaler_y_path)

    # Plot loss curves
    plot_loss(history, loss_png, title=f"Loss: {region}/{probe} (window={lookback}, horizon={horizon})")

    # ----------------------------
    # Test evaluation
    # ----------------------------
    y_pred_test_scaled = model.predict(X_test, verbose=0)          # (N, horizon)
    y_true = inverse_y_vector(scaler_y, y_test)                    # (N, horizon)
    y_pred = inverse_y_vector(scaler_y, y_pred_test_scaled)        # (N, horizon)

    # Overall metrics on flattened values
    overall = metrics_1d(y_true.reshape(-1), y_pred.reshape(-1))
    print(f"[TEST overall] MAE={overall['MAE']:.6f} RMSE={overall['RMSE']:.6f} MAPE={overall['MAPE_%']:.2f}%")

    # Save test comparison CSV with all steps
    rows = []
    d_test_dt = pd.to_datetime(d_test)
    for i in range(len(d_test_dt)):
        row = {"forecast_start_date": d_test_dt[i]}
        for s in range(horizon):
            row[f"true_t+{s+1}"] = y_true[i, s]
            row[f"pred_t+{s+1}"] = y_pred[i, s]
        rows.append(row)
    pd.DataFrame(rows).to_csv(test_cmp_csv, index=False)

    # Plot test curves for step 1 and step H (aligned by forecast start date)
    plot_series(
        dates=d_test_dt,
        y_true=y_true[:, 0],
        y_pred=y_pred[:, 0],
        out_png=test_step1_png,
        title=f"Test True vs Pred (t+1): {region}/{probe} | window={lookback}",
        ylabel="sm_30cm (m3/m3)"
    )
    plot_series(
        dates=d_test_dt,
        y_true=y_true[:, horizon - 1],
        y_pred=y_pred[:, horizon - 1],
        out_png=test_stepH_png,
        title=f"Test True vs Pred (t+{horizon}): {region}/{probe} | window={lookback}",
        ylabel="sm_30cm (m3/m3)"
    )

    # ----------------------------
    # Last-date multi-day forecast
    # ----------------------------
    df_probe_sorted = df_probe.sort_values(DATE_COL).reset_index(drop=True)
    last_date = pd.to_datetime(df_probe_sorted[DATE_COL].iloc[-1])

    last_window = df_probe_sorted[FEATURE_COLS].iloc[-lookback:].values.astype(np.float32)
    X_last = scaler_x.transform(last_window).reshape(1, lookback, 1, len(FEATURE_COLS), 1)

    y_last_scaled = model.predict(X_last, verbose=0)                # (1, horizon)
    y_last = inverse_y_vector(scaler_y, y_last_scaled)[0]           # (horizon,)

    future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=horizon, freq="D")
    forecast_df = pd.DataFrame({"date": future_dates, "pred_sm_30cm": y_last})
    forecast_df.to_csv(last_forecast_csv, index=False)

    plt.figure()
    plt.plot(future_dates, y_last, label="forecast")
    plt.xlabel("date")
    plt.ylabel("sm_30cm (m3/m3)")
    plt.title(f"Forecast next {horizon} days from last date: {region}/{probe} | window={lookback}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(last_forecast_png, dpi=160)
    plt.close()

    # Save summary metrics
    with open(summary_json, "w", encoding="utf-8") as f:
        json.dump({
            "region": region,
            "probe": probe,
            "window": lookback,
            "horizon": horizon,
            "test_overall_metrics": overall,
            "last_date": str(last_date.date()),
            "forecast_start": str((last_date + pd.Timedelta(days=1)).date()),
        }, f, ensure_ascii=False, indent=2)

    print(f"Saved outputs: {out_dir}")




In [30]:
# ============================================================
# Main: run all windows for all CSVs and all probes
# ============================================================
if __name__ == "__main__":
    REGION_CSVS: Dict[str, str] = {
        "Grandvillers_Sec": r"D:\UV Projet\Soil Moisture\Grandvillers_Sec.csv",
        "Grandvillers_Canon": r"D:\UV Projet\Soil Moisture\Grandvillers-Canon.csv",
        "Grandvillers_Robot_20": r"D:\UV Projet\Soil Moisture\Grandvillers-Robot-20.csv",
        "Grandvillers_Robot": r"D:\UV Projet\Soil Moisture\Grandvillers-Robot.csv",
    }

    OUT_ROOT = "outputs_multistep_context"

    all_probes = set()

    for region, path in REGION_CSVS.items():
        if not os.path.exists(path):
            print(f"[SKIP] File not found: {path}")
            continue

        df = load_csv(path)

        probes = sorted(df[PROBE_COL].dropna().unique().tolist())
        all_probes.update(probes)

        print(f"\nRegion={region} | probes={probes} (count={len(probes)})")

        for probe in probes:
            df_probe = df[df[PROBE_COL] == probe].copy()

            for w in WINDOWS:
                try:
                    run_one_setting(region, probe, df_probe, lookback=w, horizon=HORIZON, out_root=OUT_ROOT)
                except Exception as e:
                    print(f"[SKIP] {region}/{probe} window={w} horizon={HORIZON} -> {e}")

    print("\nAll unique probe_name values across 4 CSV:")
    print(sorted(list(all_probes)))
    print(f"Total unique probe_name: {len(all_probes)}")



Region=Grandvillers_Sec | probes=['Sec'] (count=1)

=== TRAIN: window=14 horizon=7 | Grandvillers_Sec/Sec ===
Samples: train=60, val=11, test=12
Epoch 1/150
2/2 [==============================] - 0s 111ms/step - loss: 1.4969 - val_loss: 0.7680 - lr: 0.0010
Epoch 2/150
2/2 [==============================] - 0s 20ms/step - loss: 0.7577 - val_loss: 0.7776 - lr: 0.0010
Epoch 3/150
2/2 [==============================] - 0s 18ms/step - loss: 0.6006 - val_loss: 0.7885 - lr: 0.0010
Epoch 4/150
2/2 [==============================] - 0s 19ms/step - loss: 0.4799 - val_loss: 0.7991 - lr: 0.0010
Epoch 5/150
2/2 [==============================] - 0s 17ms/step - loss: 0.3619 - val_loss: 0.8066 - lr: 0.0010
Epoch 6/150
2/2 [==============================] - 0s 16ms/step - loss: 0.3022 - val_loss: 0.8134 - lr: 0.0010
Epoch 7/150
2/2 [==============================] - 0s 16ms/step - loss: 0.2667 - val_loss: 0.8207 - lr: 0.0010
Epoch 8/150
2/2 [==============================] - 0s 18ms/step - loss: 0.23

## We evaluated ConvLSTM models with different temporal window sizes (14, 30, and 60 days) for multi-step soil moisture forecasting at 30 cm depth. Across all window configurations, the models consistently exhibited overfitting behavior, characterized by a rapid decrease in training loss while validation loss remained stable or increased. This phenomenon is primarily attributed to the limited temporal sample size, the high autocorrelation of soil moisture time series, and the relatively high model capacity of ConvLSTM compared to the available data. Increasing the temporal window did not improve generalization performance, suggesting that longer historical contexts introduce redundant information rather than additional predictive power. Overall, smaller temporal windows (e.g., 14 days) provided more stable test performance and were therefore considered more suitable for this dataset.

window=14/30/60 全部过拟合

原因是：

数据太短

土壤水分高度自相关

ConvLSTM 太强

window 变大没有带来额外信息

小 window 反而更稳、更合理

## 改进 ： 改成小 window，filters = 8 or 16, Dropout(0.2)。在训练时，随机“关闭”一部分神经元，防止模型过度依赖某些特征